In [1]:
#Import libraries
import torch
import torch.nn as nn
from skimage.io import imread, imsave
import numpy as np
import time
import os
import cv2
import bm3d
from skimage import io, img_as_float, color
from PIL import Image
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#DnCnn class imported from the github link
class DnCNN(nn.Module):
    def __init__(self, depth=17, n_channels=64, image_channels=1, use_bnorm=True, kernel_size=3):
        super(DnCNN, self).__init__()
        kernel_size = 3
        padding = 1
        layers = []
        layers.append(nn.Conv2d(in_channels=image_channels, out_channels=n_channels,
                      kernel_size=kernel_size, padding=padding, bias=True))
        layers.append(nn.ReLU(inplace=True))
        for _ in range(depth-2):
            layers.append(nn.Conv2d(in_channels=n_channels, out_channels=n_channels,
                          kernel_size=kernel_size, padding=padding, bias=False))
            layers.append(nn.BatchNorm2d(
                n_channels, eps=0.0001, momentum=0.95))
            layers.append(nn.ReLU(inplace=True))
        layers.append(nn.Conv2d(in_channels=n_channels, out_channels=image_channels,
                      kernel_size=kernel_size, padding=padding, bias=False))
        self.dncnn = nn.Sequential(*layers)
        self._initialize_weights()

    def forward(self, x):
        y = x
        out = self.dncnn(x)
        return y-out

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.orthogonal_(m.weight)
                print('init weight')
                if m.bias is not None:
                    init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                init.constant_(m.weight, 1)
                init.constant_(m.bias, 0)

In [3]:
#Inference function that denoises the image using the model provided
def infer(model, image_path, sigma):
    x = np.array(imread(image_path),
                 dtype=np.float32)/255.0
    np.random.seed(seed=0)  # for reproducibility
    # Add Gaussian noise without clipping
    y = x + np.random.normal(0, sigma/255.0, x.shape)
    y = y.astype(np.float32)
    y_ = torch.from_numpy(y).view(1, -1, y.shape[0], y.shape[1])

    y_ = y_.cpu()
    x_ = model(y_)  # inference
    x_ = x_.view(y.shape[0], y.shape[1])
    x_ = x_.cpu()
    x_ = x_.detach().numpy().astype(np.float32)

    return x_

In [4]:
#This function is used to calculate the PSNR and SSIM
def calcMetrics(img1, img2):
    psnr = peak_signal_noise_ratio(img1, img2)
    ssim = structural_similarity(img1, img2)
    return psnr, ssim


if __name__ == "__main__":
    #Specifying the paths
    IMG_PATH = "images/OrigRGB_Img.png"
    GROUND_TRUTH_IMG = 'images/GROUND_TRUTH.png'
    MODEL_PATH = "model.pth"
    SIGMA = 25
    SAVE_DIR = "results"
    RESULT_IMG_NAME = 'denoise_dnn.png'

    #Convert the real image to greyscale for use
    img = cv2.imread(IMG_PATH, 0)
    cv2.imwrite("images/OrigGray_Img.png", img)
    
    IMG_PATH = "images/OrigGray_Img.png"

    #Loading the pretrained weights into the DnCnn model
    model = torch.load(MODEL_PATH, map_location=torch.device('cpu'))
    model.eval()

    #Running the inference on the image
    result = infer(model, IMG_PATH, SIGMA)

    #Making the directory and saving out image in it
    os.makedirs(SAVE_DIR, exist_ok=True)
    SAVE_PATH = os.path.join(SAVE_DIR, RESULT_IMG_NAME)
    imsave(SAVE_PATH, np.clip(result, 0, 1))

    #Importing the true image
    truth_img = cv2.imread(GROUND_TRUTH_IMG, 0)

    #Copied ASIS from task1 for comparison only
    img = cv2.imread(IMG_PATH, cv2.IMREAD_COLOR)
    denoise_nlmean = cv2.fastNlMeansDenoisingColored(img,None,10,10,7,21)
    cv2.imwrite('results/denoise_nlmean.png',denoise_nlmean)

    img = img_as_float(io.imread("images/OrigRGB_Img.png"))
    denoised_bm3d = bm3d.bm3d(img, sigma_psd=0.2, stage_arg=bm3d.BM3DStages.HARD_THRESHOLDING)
    plt.imsave("results/denoise_bm3d.png", denoised_bm3d)

    #Reading the required images as greyscale
    img1 = cv2.imread('results/denoise_dnn.png', 0)
    img2 = cv2.imread('results/denoise_nlmean.png', 0)
    img3 = cv2.imread("results/denoise_bm3d.png", 0)

    #calculate the psnr and ssim
    psnr_dn, ssim_dn = calcMetrics(truth_img, img1)
    psnr_nlmean, ssim_nlmean = calcMetrics(truth_img, img2)
    psnr_bm3d, ssim_bm3d = calcMetrics(truth_img, img3)

    #Converting our results to a dataframe and saving in file
    data={'PSNR':[psnr_dn,psnr_nlmean,psnr_bm3d],'SSIM':[ssim_dn,ssim_nlmean,ssim_bm3d]}
    df=pd.DataFrame(data,index=['DnCnn','NL Mean','BM3D'])
    df.to_csv('results/Results.csv')


/Users/nuzhatnishi/opt/anaconda3/lib/python3.8/site-packages/torch/serialization.py:637: UserWarning: Couldn't retrieve source code for container of type DnCNN. It won't be checked for correctness upon loading.
  warnings.warn("Couldn't retrieve source code for container of "
/Users/nuzhatnishi/opt/anaconda3/lib/python3.8/site-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Users/nuzhatnishi/opt/anaconda3/lib/python3.8/site-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patc